In [99]:
import os
import geopandas as gpd
import folium
from shapely.geometry import Polygon, Point
import json

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
filepath = "/content/drive/MyDrive/Data Hackathon_IA_2025/couche/"

# Liste des fichiers .geojson
geojson_files = [file for file in os.listdir(filepath) if file.endswith(".geojson")]

geo_layers = {}
for file in geojson_files:
    full_path = os.path.join(filepath, file)
    geo_layers[file.replace(".geojson", "")] = gpd.read_file(full_path)

In [102]:


def convert_datetime_columns_to_str(gdf):
    datetime_cols = gdf.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]']).columns
    for col in datetime_cols:
        gdf[col] = gdf[col].astype(str)
    return gdf

In [103]:
andf_layers = ['aif', 'air_proteges',
              'dpl', 'dpm', 'enregistrement individuel',
              'litige', 'parcelles', 'restriction', 'tf_demembres',
              'tf_en_cours', 'tf_etat', 'titre_reconstitue', 'zone_inondable']

# Filtrage des couches existantes
andf_layers = {layer_name: layer for layer_name, layer in geo_layers.items() if layer_name in andf_layers}

In [104]:
coords = [
    {"x": 414685.079000000027008, "y": 713937.121000000042841},
    {"x":414728.347, "y":713934.22199999995064} ,
    {"x":414742.634000000020023, "y":713867.503999999957159} ,
    {"x":414773.692000000039116,"y": 713765.42699999990873} ,
    {"x": 414785.441, "y":713715.569999999948777} ,
    {"x": 414812.713999999978114, "y":713663.979000000050291} ,
    {"x":414679.330000000016298, "y":713646.162999999942258} ,
    {"x": 414657.23599999997532, "y":713648.562000000034459} ,
    {"x": 414636.550144485081546,"y": 713647.810810075956397} ,
    {"x": 414625.981655898038298, "y":713727.238676203531213} ,
    {"x": 414622.308431876008399, "y":713772.697338390629739 },
    {"x": 414576.46544485760387, "y":713765.425226306193508} ,
    {"x": 414520.470278906868771, "y":713774.298431224073283} ,
    {"x": 414530.927716866368428, "y":713782.135254827793688} ,
    {"x": 414571.474854857195169, "y":713799.585319338249974} ,
    {"x": 414631.612989722169004, "y":713841.898917965707369 },
    {"x": 414645.367546415072866,"y": 713859.924566888366826} ,
    {"x": 414645.060030365246348, "y":713928.252450411091559} ,
    {"x":414629.653771227633115, "y":713924.820584255154245} ,
    {"x": 414635.878000000026077, "y":713930.86199999996461 },
    {"x": 414685.079000000027008, "y":713937.121000000042841}
]


points = [(c["x"], c["y"]) for c in coords]
parcel = Polygon(points)



eligibility = {}
for name, layer in andf_layers.items():
    eligibility[name] = layer.intersects(parcel).any()

print("Couches intersectant avec le terrain :", eligibility)

Couches intersectant avec le terrain : {'litige': np.True_, 'air_proteges': np.False_, 'enregistrement individuel': np.False_, 'dpm': np.False_, 'aif': np.False_, 'dpl': np.False_, 'restriction': np.False_, 'parcelles': np.True_, 'tf_demembres': np.False_, 'zone_inondable': np.False_, 'tf_en_cours': np.False_, 'tf_etat': np.False_, 'titre_reconstitue': np.False_}


In [105]:

parcel_gdf = gpd.GeoSeries([parcel], crs=SCR_SOURCE)
parcel_wgs84 = parcel_gdf.to_crs(epsg=4326).iloc[0]

center_point_wgs84 = parcel_wgs84.centroid
center_lat, center_lon = center_point_wgs84.y, center_point_wgs84.x


points_wgs84_folium = [(lat, lon) for lon, lat in parcel_wgs84.exterior.coords]


In [106]:
# m = folium.Map(location=[center_lat, center_lon], zoom_start=13, prefer_canvas=True)
m = folium.Map(location=[center_lat, center_lon], zoom_start=18, prefer_canvas=True) # J'ai augmenté le zoom pour mieux voir


In [107]:
layer_colors = {
    "aif": "#006400",
    "air_proteges": "#228B22",
    "dpl": "#1E90FF",
    "dpm": "#00CED1",
    "enregistrement individuel": "#FFD700",
    "litige": "#FF0000",
    "parcelles": "#8B4513",
    "restriction": "#FF8C00",
    "tf_demembres": "#9932CC",
    "tf_en_cours": "#00FF7F",
    "tf_etat": "#4169E1",
    "titre_reconstitue": "#DC143C",
    "zone_inondable": "#87CEFA"
}


In [108]:

def filter_layer_by_distance(gdf, center_point, max_dist):
    # center_point ici doit être le centroïde NON-corrigé (parcel.centroid)
    return gdf[gdf.geometry.distance(center_point) <= max_dist]

center_point = parcel.centroid
radius_meters = 5000

# Boucle pour itérer sur les couches
for layer_name, gdf in andf_layers.items():

    # 1. Assurez-vous que le CRS source est défini
    if gdf.crs is None:
        gdf.set_crs(SCR_SOURCE, inplace=True)

    # 2. Filtrer la couche
    filtered_gdf = filter_layer_by_distance(gdf, center_point, radius_meters)

    if filtered_gdf.empty:
        continue

    # 3. Transformer en WGS 84 (EPSG:4326) pour Folium
    filtered_gdf_wgs84 = filtered_gdf.to_crs(epsg=4326)

    # Assurez-vous que convert_datetime_columns_to_str est défini
    filtered_gdf_str_converted = convert_datetime_columns_to_str(filtered_gdf_wgs84)
    geojson_str = filtered_gdf_str_converted.to_json()
    geojson_data = json.loads(geojson_str)

    color = layer_colors.get(layer_name, "gray")

    # --- DÉFINITION DYNAMIQUE DU POP-UP ---
    popup_fields = None

    if layer_name == "tf_demembres":
        # Pop-up pour le numéro du Titre Foncier Démembré
        popup_fields = folium.GeoJsonPopup(
            fields=['num_tf'],
            aliases=['Numéro du Titre Foncier:'],
            localize=True
        )
    elif layer_name == "restriction":
        # Pop-up pour les restrictions (CORRECTION DE L'ORTHOGRAPHE : 'designation')
        popup_fields = folium.GeoJsonPopup(
            fields=['type', 'designation'],
            aliases=['Type de Restriction:', 'Détails:'],
            localize=True
        )

    # --- AJOUT DE LA COUCHE CORRIGÉE ---
    folium.GeoJson(
        geojson_data,
        name=layer_name, # CORRECTION : Nom de la couche pour le LayerControl
        popup=popup_fields,
        style_function=lambda feature, col=color: {
            'fillColor': col,
            'color': col,
            'weight': 2,
            'fillOpacity': 0.4
        }
    ).add_to(m)

# --- AJOUT DU POLYGONE DE VOTRE TERRAIN (Doit être en WGS 84) ---
points_wgs84_folium = [(lat, lon) for lon, lat in parcel_wgs84.exterior.coords]

folium.Polygon(
    locations=points_wgs84_folium,
    color='blue',
    weight=3,
    fill=True,
    fill_opacity=0.2,
    popup='Terrain utilisateur'
).add_to(m)

# --- AJOUT DU CONTRÔLE DES COUCHES ---
folium.LayerControl().add_to(m)

# Affiche la carte finale
m

Output hidden; open in https://colab.research.google.com to view.